***PRIYANSHU KHARVA***

**Stock Market Prediction And Forecasting Using Stacked LSTM **

**DATASET LINK :- https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv **

**ALGORITHM USED :- Sklearn inverse scalar prediction**

Importing Libraries

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import ensemble, metrics
import xgboost as xgb
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1.0


In [ ]:
df.shape

(7768, 6)

In [ ]:
df = df.sample(frac=0.1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 777 entries, 6261 to 2066
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   msno                777 non-null    object 
 1   song_id             777 non-null    object 
 2   source_system_tab   777 non-null    object 
 3   source_screen_name  766 non-null    object 
 4   source_type         777 non-null    object 
 5   target              777 non-null    float64
dtypes: float64(1), object(5)
memory usage: 42.5+ KB


In [ ]:
songs = pd.read_csv('songs.csv')

In [ ]:
songs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 777 entries, 6261 to 2066
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   msno                777 non-null    object 
 1   song_id             777 non-null    object 
 2   source_system_tab   777 non-null    object 
 3   source_screen_name  766 non-null    object 
 4   source_type         777 non-null    object 
 5   target              777 non-null    float64
dtypes: float64(1), object(5)
memory usage: 42.5+ KB


In [ ]:
df1_close = df1['Close']
df1_close

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 777 entries, 0 to 776
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   msno                777 non-null    object 
 1   song_id             777 non-null    object 
 2   source_system_tab   777 non-null    object 
 3   source_screen_name  766 non-null    object 
 4   source_type         777 non-null    object 
 5   target              777 non-null    float64
 6   song_length         363 non-null    float64
 7   genre_ids           355 non-null    object 
 8   artist_name         363 non-null    object 
 9   composer            288 non-null    object 
 10  lyricist            232 non-null    object 
 11  language            363 non-null    float64
dtypes: float64(3), object(9)
memory usage: 78.9+ KB


In [ ]:
df1_close = scaler.fit_transform(np.array(df1_close).reshape(-1,1))
df1_close

array([[0.16584967],
       [0.16319444],
       [0.1621732 ],
       ...,
       [0.62622549],
       [0.62214052],
       [0.62418301]])

In [ ]:
training_size = int(len(df1_close)*0.70)
test_size = len(df1_close) - training_size
train, test = df1_close[0: training_size,:], df1_close[training_size : len(df1_close),:1]

In [ ]:
def create_mat(ds, time_step = 1):
    dataX, dataY = [], []
    for i in range(len(ds)-time_step-1):
        a = ds[i:(i+time_step),0]
        dataX.append(a)
        dataY.append(ds[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 100
X_train, y_train = create_mat(train, time_step)
X_test, y_test = create_mat(test, time_step)

In [ ]:
X_train.shape

(1323, 100)

In [ ]:
y_train.shape

(1323,)

In [ ]:
X_test.shape

(510, 100)

In [ ]:
y_test.shape

(510,)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100, 50)           10400     
                                                                 
 lstm_4 (LSTM)               (None, 100, 50)           20200     
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, validation_split=0.1, epochs= 60, batch_size=64, verbose = 1)

Epoch 1/60
19/19 [==============================] - 10s 254ms/step - loss: 0.0101 - val_loss: 0.0013
Epoch 2/60
19/19 [==============================] - 3s 180ms/step - loss: 0.0017 - val_loss: 0.0014
Epoch 3/60
19/19 [==============================] - 3s 177ms/step - loss: 9.0100e-04 - val_loss: 0.0012
Epoch 4/60
19/19 [==============================] - 3s 179ms/step - loss: 8.6627e-04 - val_loss: 0.0012
Epoch 5/60
19/19 [==============================] - 4s 185ms/step - loss: 8.3909e-04 - val_loss: 0.0011
Epoch 6/60
19/19 [==============================] - 4s 188ms/step - loss: 8.1775e-04 - val_loss: 0.0011
Epoch 7/60
19/19 [==============================] - 4s 185ms/step - loss: 7.9463e-04 - val_loss: 0.0010
Epoch 8/60
19/19 [==============================] - 3s 184ms/step - loss: 7.6136e-04 - val_loss: 9.8522e-04
Epoch 9/60
19/19 [==============================] - 3s 184ms/step - loss: 7.4990e-04 - val_loss: 9.4360e-04
Epoch 10/60
19/19 [==============================] - 4s 185ms/s

In [ ]:
predictions = model.predict(X_test)

In [ ]:
inv_predictions = scaler.inverse_transform(predictions)

In [ ]:
inv_predictions

array([[141.5544  ],
       [141.72646 ],
       [140.79953 ],
       [138.0814  ],
       [136.38162 ],
       [135.8778  ],
       [137.14688 ],
       [138.81989 ],
       [139.24364 ],
       [138.82077 ],
       [138.50261 ],
       [139.65858 ],
       [140.46997 ],
       [142.05266 ],
       [144.26588 ],
       [142.21408 ],
       [139.65398 ],
       [139.21971 ],
       [139.91724 ],
       [145.08632 ],
       [149.66653 ],
       [151.81613 ],
       [151.98236 ],
       [149.69592 ],
       [149.34337 ],
       [149.14442 ],
       [149.63663 ],
       [151.43143 ],
       [152.24817 ],
       [152.2352  ],
       [152.56364 ],
       [151.36127 ],
       [148.05028 ],
       [142.28679 ],
       [138.81671 ],
       [137.94846 ],
       [138.31798 ],
       [137.67763 ],
       [135.39543 ],
       [131.64482 ],
       [129.02567 ],
       [127.80549 ],
       [125.76403 ],
       [124.937225],
       [122.932625],
       [120.307655],
       [119.54191 ],
       [119.2

In [ ]:
mse = math.sqrt(mean_squared_error(y_test, predictions))
print("The Mean Squared Error for above model is: ", mse)

The Mean Squared Error for above model is:  0.0403524734798987
